# Actividad Ensambles

In [ ]:
import pandas as pd

In [76]:
df = pd.read_csv('properati_ml.csv')

In [77]:
df.head()

price_usd        lat        lon  surface_total  surface_scaled  \
0   119000.0 -34.615181 -58.385944           56.0        0.045249   
1   180000.0 -34.613748 -58.377643          104.0        0.099548   
2   255000.0 -34.614589 -58.383963          104.0        0.099548   
3   189900.0 -34.611730 -58.378348           83.0        0.075792   
4   135947.0 -34.616651 -58.390361           51.0        0.039593   

   floor_replaced  expenses_replaced  surface_covered  rooms  rooms_stand  \
0        9.000000             3000.0             56.0    2.0    -0.578163   
1       11.327957             3000.0             96.0    4.0     0.803576   
2       11.327957             3000.0             94.0    4.0     0.803576   
3       13.000000             3000.0             78.0    4.0     0.803576   
4       11.327957             3000.0             51.0    2.0    -0.578163   

   apartment  house  
0          1      0  
1          1      0  
2          1      0  
3          1      0  
4          1      0

In [78]:
import numpy as np
from sklearn.model_selection import train_test_split

X = df.drop(['price_usd'], axis=1)
y = df[['price_usd']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## Primer modelo

1- Entrenar un algoritmo para predecir 'price_usd' usando todas las variables con una regresión lineal. Llamemos al estimador est_1.

In [79]:
import numpy as np
from sklearn.linear_model import LinearRegression

est_1 = LinearRegression()
est_1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

2- Calcular el RMSE de test para est_1.

In [80]:
from sklearn.metrics import mean_squared_error
price_usd_est_1 = est_1.predict(X_test)
mse = mean_squared_error(y_test, price_usd_est_1)
rmse = np.sqrt(mse)
print(rmse)

68834.25454785179


## Segundo modelo

3- Entrenar un algoritmo para predecir 'price_usd' usando todas las variables con una regresión rigde. Llamemos al estimador est_2.

In [81]:
import numpy as np
from sklearn.linear_model import Ridge

est_2 = Ridge(alpha=1.0)
est_2.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

4- Calcular el RMSE de test para est_2.

In [82]:
price_usd_est_2 = est_2.predict(X_test)
mse = mean_squared_error(y_test, price_usd_est_2)
rmse = np.sqrt(mse)
print(rmse)

69019.3271116377


## Tercer modelo

5- Entrenar un algoritmo para predecir 'price_usd' usando todas las variables con un árbol de decisión con profundidad parámetros default. Llamemos al estimador est_3.

In [83]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

est_3 = DecisionTreeRegressor()
est_3.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

6- Calcular el RMSE de test para est_3.

In [84]:
price_usd_est_3 = est_3.predict(X_test)
mse = mean_squared_error(y_test, price_usd_est_3)
rmse = np.sqrt(mse)
print(rmse)

75387.5583287745


## Ensamble

7- Agrupar las predicciones en testing de est_1, est_2 y est_3 en un DataFrame (df_preds).

In [85]:
print(price_usd_est_1.shape, price_usd_est_2.shape, price_usd_est_3.shape)

(2790, 1) (2790, 1) (2790,)


In [86]:
price_usd_est_3 = price_usd_est_3.reshape(2790,1)

In [87]:
preds = np.concatenate([price_usd_est_1, price_usd_est_2, price_usd_est_3], axis=1)
df_preds = pd.DataFrame(preds)
df_preds.columns = ['price_usd_est_1', 'price_usd_est_2', 'price_usd_est_3']
df_preds.head()

price_usd_est_1  price_usd_est_2  price_usd_est_3
0    103653.253125    111440.762303          86800.0
1    523193.508409    521999.878223         525000.0
2    130432.908673    121958.178921         149000.0
3    262667.419634    260806.798207         235000.0
4    160829.385471    158132.181665         150000.0

In [88]:
#ojo los indices
y_test.head().reset_index()

index  price_usd
0   5006   136000.0
1   5432   525000.0
2    625   124000.0
3   3396   380000.0
4   7169   142000.0

8- Agregar a df_preds los 'price_usd' de test.

La estructura del DataFrame debería ser 'price_usd, price_usd_est_1, price_usd_est_2, price_usd_est_3'

In [89]:
y_test = y_test.reset_index(drop=True)
df_preds = pd.concat([y_test, df_preds], axis=1)
df_preds.columns = ['price_usd', 'price_usd_est_1', 'price_usd_est_2', 'price_usd_est_3']
df_preds.head()

price_usd  price_usd_est_1  price_usd_est_2  price_usd_est_3
0   136000.0    103653.253125    111440.762303          86800.0
1   525000.0    523193.508409    521999.878223         525000.0
2   124000.0    130432.908673    121958.178921         149000.0
3   380000.0    262667.419634    260806.798207         235000.0
4   142000.0    160829.385471    158132.181665         150000.0

9- Crear una columna 'price_usd_ensamble' que tenga el promedio de price_usd_est_1, price_usd_est_2 y price_usd_est_3.

In [90]:
df_preds['price_usd_ensamble_mean'] = df_preds[['price_usd_est_1', 'price_usd_est_2', 'price_usd_est_3']].mean(axis=1)
#alternativa
#df_preds['price_usd_ensamble'] = df_preds[['price_usd_est_1', 'price_usd_est_2', 'price_usd_est_3']].apply(lambda x,: np.mean(x),axis=1)
df_preds.head()

price_usd  price_usd_est_1  price_usd_est_2  price_usd_est_3  \
0   136000.0    103653.253125    111440.762303          86800.0   
1   525000.0    523193.508409    521999.878223         525000.0   
2   124000.0    130432.908673    121958.178921         149000.0   
3   380000.0    262667.419634    260806.798207         235000.0   
4   142000.0    160829.385471    158132.181665         150000.0   

   price_usd_ensamble_mean  
0            100631.338476  
1            523397.795544  
2            133797.029198  
3            252824.739280  
4            156320.522379

10- Calcular el RMSE para el ensamble.

In [91]:
mse = mean_squared_error(df_preds.price_usd, df_preds.price_usd_ensamble_mean)
rmse = np.sqrt(mse)
print(rmse)

63371.577303248814


11- Reemplazar la media en el punto 9 por otro estadístico descriptivo (como mediana, máximo y mínimo) y calcular el RMSE.

In [92]:
df_preds['price_usd_ensamble_max'] = df_preds[['price_usd_est_1', 'price_usd_est_2', 'price_usd_est_3']].max(axis=1)
#alternativa
#df_preds['price_usd_ensamble'] = df_preds[['price_usd_est_1', 'price_usd_est_2', 'price_usd_est_3']].apply(lambda x,: np.mean(x),axis=1)
df_preds.head()

price_usd  price_usd_est_1  price_usd_est_2  price_usd_est_3  \
0   136000.0    103653.253125    111440.762303          86800.0   
1   525000.0    523193.508409    521999.878223         525000.0   
2   124000.0    130432.908673    121958.178921         149000.0   
3   380000.0    262667.419634    260806.798207         235000.0   
4   142000.0    160829.385471    158132.181665         150000.0   

   price_usd_ensamble_mean  price_usd_ensamble_max  
0            100631.338476           111440.762303  
1            523397.795544           525000.000000  
2            133797.029198           149000.000000  
3            252824.739280           262667.419634  
4            156320.522379           160829.385471

In [94]:
mse = mean_squared_error(df_preds.price_usd, df_preds.price_usd_ensamble_max)
rmse = np.sqrt(mse)
print(rmse)

73086.60992415181


12- Cuál fue el mejor modelo?

El ensamble con promedio!